# Ejercicio a Entregar   Data Frame


## Ejercicio1 - Enumera y describe al menos 5 maneras diferentes de crear un DataFame

In [ ]:

#Método 1 desde una lista de columnas sin esquemas
df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
#Método 2 desde una lista de columnas con esquema
#Haríamos lo mismo pasando el método schema como parámetro
#el formato es variable tipo, variable tipo,


#Método 3 desde un df de pandas
#método createDataFrame y pasar el dataframe de pandas, hay que definir que los items del df son iteritems

# método 4 desde un RDD
#createDataFrame a un objeto RDD

#método 5 desde un RDD con toDF
#desde un rdd usando la funcion toDF() y pasandole como parametros el nombre de las columnas

## Ejercicio2 - ¿Se puede crear un Dataframe desde una Base de datos?

In [ ]:
#Ejercicio 2 -respuesta:
#Sí, podemos crear un dataframe a partir de una base de datos utilizando el api de SQL que viene por defecto en
#pyspark, con esta podemos crear dataframes a partir de la mayoría de bases de datos relacionales y no relacionales



## Ejercicio 3- A la hora de crear un dataframe desde un fichero, ¿que diferencia existe entre las siguientes líneas?

df = spark.read.options(header='True',inferSchema='True').json("stranger.json")

df = spark.read.json("stranger.json")

            


In [ ]:
#Ejercicio 3 -respuesta .....

#La primera está preformateando de tal manera que está diciendo de antemano que el dataframe tiene una
#fila de cabecera y que infiera el esquema del archivo
#En la segunda no hace nada de esto, lee el archivo tal cual

## Ejercicio 4- ¿por qué con ambos métodos se crea el mismo esquema?

df.printSchema()


root
 |-- edad: long (nullable = true)
 |-- nombre: string (nullable = true)
 |-- sexo: string (nullable = true)


Ejercicio 4 -respuesta .....  






## Ejercicio 5- Dado el fichero de stranger obtener la cantidad de hombres y mujeres usando data frame

In [5]:
#Ejercicio 5 -codigo
df = spark.read.options(header='True', inferSchema='True').json("stranger.json")

countWomen = df.select('nombre').where("sexo='M'").count()
countMen = df.select('nombre').where("sexo='H'").count()
print(countWomen)
print(countMen)

4
6


Hay 4 mujeres y 6 hombres

## Ejercicio 6- Teniendo cargado el fichero stranger.txt  (este fichero se encuentra en el zip de la sesion2),
Al cargarlo obtenemos una fila por cada autor pero en vez de tener 3 columnas, carga cada fila como un array.

¿cómo hacemos para obtener sus 3 columnas?

In [6]:
dftxt = spark.read.text("stranger.txt")
dftxt.show()

+---------------+
|          value|
+---------------+
|  Eleven, 12, M|
|  Hopper, 45, H|
|    Will, 11, H|
|     Max, 11, M|
|    Mike, 11, H|
|   Lucas, 11, H|
|  Dustin, 11, H|
|  Eleven, 12, M|
|   Nancy, 23, M|
|Jonathan, 24, H|
+---------------+



In [7]:
#ejercio 6 completar

#pista --> https://sparkbyexamples.com/spark/spark-read-text-file-rdd-dataframe/#dataframe-text
def partir(fila):
    elements = fila.split(", ")
    return elements                                   
 
#print(partir("hola , fff ,ffff"))            
          
    
rdd1 = dftxt.rdd  
#rdd1.collect()
rddfile = rdd1.map(lambda x: partir(x[0]))
#rddfile.collect()
rddfile.toDF(schema=['nombre', 'edad', 'sexo']).show()

+--------+----+----+
|  nombre|edad|sexo|
+--------+----+----+
|  Eleven|  12|   M|
|  Hopper|  45|   H|
|    Will|  11|   H|
|     Max|  11|   M|
|    Mike|  11|   H|
|   Lucas|  11|   H|
|  Dustin|  11|   H|
|  Eleven|  12|   M|
|   Nancy|  23|   M|
|Jonathan|  24|   H|
+--------+----+----+



# Ejercios sobre la tienda virtual

Los datos de la tienda virtual se descargaron en la clase anterior desde el sitio: https://www.kaggle.com/mkechinov/ecommerce-events-history-in-cosmetics-shop

Nota: no se deben volver a descargar los archivos.



In [10]:
#cargamos los datos de la tienda virtual


dfTienda = spark.read.options(header='True', inferSchema='True').csv("./archive(3)/*.csv")



## Ejercicio 7 :  ¿Obtener las 2 sesiones  en las que menos productos diferentes se hayan comprado? Mostar las dos sesiones junto con el número  productos diferentes comprados en cada una de ellas

Nota=  filtar por event_type = purchase

        

In [32]:
leastBoughtSessions = dfTienda.filter("event_type='purchase'").groupBy("user_session").count()
leastBoughtSessions.orderBy("count",ascending=True).show(2)

+--------------------+-----+
|        user_session|count|
+--------------------+-----+
|b2167f2d-7a93-4c0...|    1|
|6a541a8c-1e17-466...|    1|
+--------------------+-----+
only showing top 2 rows



#Ejercicio 8:  Dadas las 3 marcas más polulares de la tienda
1- obtener el numero de usuarios que han comprado alguna de esas marcas

2) obtener el nombre/id de los usuarios que han comprado alguna de esas marcas

3) obtener el total de dinero que se ha vendido de cada una de las marcas

In [34]:
marcas= df.filter("event_type='purchase'").groupBy("brand").count()
marcas.orderBy("count", ascending=False).show(3)

+------+------+
| brand| count|
+------+------+
|  null|549693|
|runail|111408|
| irisk| 73806|
+------+------+
only showing top 3 rows



In [38]:
boughtBrandUser = df.select('user_id').filter("event_type='purchase' and brand = 'null' or brand = 'runail' or brand= 'irisk'").distinct().count()
print(boughtBrandUser)

331890


In [36]:
boughtBrandUser = df.select('user_id').filter("event_type='purchase' and brand = 'null' or brand = 'runail' or brand= 'irisk'")
boughtBrandUser.show()

+---------+
|  user_id|
+---------+
|494077766|
|560109803|
|494077766|
|576802932|
|576802932|
|576802932|
|494077766|
|494077766|
|579970581|
|568755469|
|576802932|
|482493001|
|544461907|
|509445281|
|494077766|
|556730134|
|494077766|
|499259978|
|513026898|
|513026898|
+---------+
only showing top 20 rows



3) obtener el total de dinero que se ha vendido de cada una de las marcas

In [41]:
productsIdWithBrand = df.select('product_id').filter("brand = 'null' or brand = 'runail' or brand= 'irisk'").distinct()

In [52]:
from pyspark.sql.functions import col

moneyWon = df.select('price').filter(col("product_id").isin(productsIdWithBrand['product_id'])).filter("event_type='purchase'").count()
print(moneyWon)

1287007
